In [1]:
import numpy as np

In [2]:
###----------EJERCICIO 1-------------------

def cargarDataset():
    base = "template-alumnos/dataset/cats_and_dogs"

    # --- TRAIN ---
    X_cats = np.load(f"{base}/train/cats/efficientnet_b3_embeddings.npy")   
    X_dogs = np.load(f"{base}/train/dogs/efficientnet_b3_embeddings.npy")   

    X_train = np.hstack([X_cats, X_dogs])  

    Y_train = np.hstack([
        np.tile([[1],[0]], (1, X_cats.shape[1])),  # gatos
        np.tile([[0],[1]], (1, X_dogs.shape[1]))   # perros
    ])                                             

    # --- VAL ---
    V_cats = np.load(f"{base}/val/cats/efficientnet_b3_embeddings.npy")
    V_dogs = np.load(f"{base}/val/dogs/efficientnet_b3_embeddings.npy")

    X_val = np.hstack([V_cats, V_dogs])
    Y_val = np.hstack([
        np.tile([[1],[0]], (1, V_cats.shape[1])),
        np.tile([[0],[1]], (1, V_dogs.shape[1]))
    ])

    print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
    print("X_val:", X_val.shape, "Y_val:", Y_val.shape)

    return X_train, Y_train, X_val, Y_val

In [16]:
Xt, Yt, Xv, Yv = cargarDataset()

Yv


X_train: (1536, 2000) Y_train: (2, 2000)
X_val: (1536, 1000) Y_val: (2, 1000)


array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1]], shape=(2, 1000))

In [ ]:
def matrizDeConfusion(Xv,W,Yv):

    y_pred = W @ Xv

    

In [18]:
## FUNCIONES PARA ELMODULO ALC / AUXILIARES

## MULTIPLICACION DE MATRICES

def matmul(A, B):

    m = len(A)
    n = len(A[0])

    
    if len(B) != n:
        return "Dimensiones incompatibles"
    
    p = len(B[0])

    # inicializo matriz C de ceros m x p
    C = []
    for _ in range(m):
        C.append([0.0] * p)

    # calculo C
    for i in range(m):
        for k in range(n):
            aik = A[i][k]
            for j in range(p):
                C[i][j] += aik * B[k][j]

    return C

def matmul_multiple(A,B,C):

    aux = matmul(A,B)

    return matmul(aux,C)

## SUSTITUCION HACIA ADELANTE
def sust_adelante(L,b):

    L = np.array(L, dtype=float)
    b = np.array(b, dtype=float)

    n = L.shape[0]
    x = np.zeros(n)

    for i in range(n):

        suma = 0
        for j in range(i):

            suma += (L[i][j]*x[j])
            
        termino = b[i] - suma
        x[i] = termino/L[i][i]

    return x

## SUSTITUCION HACIA ATRAS
def sust_atras(L, Z):

    L = np.array(L, dtype=float)
    Z = np.array(Z, dtype=float)
    
    n = L.shape[0]
    x = np.zeros(n)



    for i in range(n-1, -1, -1):
        suma = 0
        for j in range(i+1, n):

            suma += (L[i][j]*x[j])
            
        termino = Z[i] - suma

        x[i]=termino/L[i][i]


    return x


def sust_atras_matriz(U, B):
    """
    Resuelve U X = B para cuando B es matriz
   
    """

    U = np.array(U, dtype=float)
    B = np.array(B, dtype=float)

    n, m = B.shape
    X = np.zeros((n, m))

    for j in range(m):
        bj = B[:, j]
        xj = sust_atras(U, bj)
        X[:, j] = xj

    return X

def sust_adelante_matriz(L, B):
    """
    Resuelve L Z = B para cuando B es matriz

    """
    L = np.array(L, dtype=float)
    B = np.array(B, dtype=float)



    n, m = B.shape
    Z = np.zeros((n, m))

    for j in range(m):
        bj = B[:, j]
        zj = sust_adelante(L, bj)
        Z[:, j] = zj

    return Z


def cholesky(A):
    
    L = np.zeros_like(A)
    n = L.shape[0]

    for i in range(0,n):
    
    # Bucle 2 (columnas, hasta la diagonal)
        for j in range(i + 1):
        
        # --- CASO DIAGONAL (i == j) ---
            if i == j:
                suma_cuadrados = 0
            
            # Bucle 3 (para la sumatoria)
                for k in range(j):  
                    suma_cuadrados += L[i][k]**2
                
                termino = A[i][i] - suma_cuadrados
            
                if termino < 0:
                    pass
            
                L[i][i] =np.sqrt(termino)

        # --- CASO VERTICAL (i > j) ---
        else:
            producto_punto = 0
            
            # Bucle 3 (para la sumatoria)
            for k in range(j):  # k va de 0 a j-1
                producto_punto += L[i][k] * L[j][k]
            
            
            L[i][j] = (A[i][j] - producto_punto) / L[j][j]

    return L



def transpuesta(M):
    filas = len(M)
    cols = len(M[0])
    T = [[0]*filas for _ in range(cols)]
    for i in range(filas):
        for j in range(cols):
            T[j][i] = M[i][j]
    return T


def esSimetrica(A):
    return matricesIguales(A,transpuesta(A))

def todosCeros(A):

    for i in A:
        if i != 0:
            return False
        
    return True

## FALTA HACER ELIM GAUSSIANA
def rango(A):

    n = A.shape

    rango = 0

    matriz_gauss = eliminacionGaussiana(A)

    for i in matriz_gauss:
        if not todosCeros(i):
            rango +=1

    return rango

def error(x, y):
    x = np.float64(x)
    y = np.float64(y)

    return np.abs(x - y)


def matricesIguales(A, B, tol):
    
    if np.shape(A) != np.shape(B):
        return False
    
    n, m = np.shape(A)

    for i in range(n):
        for j in range(m):
            # Esta definición es equivalente a la definición de numpy.allclose
            # con la diferencia que es simétrica respecto de A y B
            if error(A[i, j], B[i, j]) > tol:
                return False

    return True

def producto_interno(u: np.array, v: np.array) -> float:
    if len(u) != len(v):
        return None
    suma = 0
    for i in range(len(u)):
        suma += u[i] * v[i]
    return suma

def norma(v):
    return np.sqrt(producto_interno(v, v))


def producto_matricial(A: np.array, B: np.array) -> np.array:
    tamA = A.shape
    mA = tamA[0]
    nA = tamA[1]

    tamB = B.shape
    mB = tamB[0]
    nB = tamB[1]

    if nA != mB:
        return None
    
    res = np.zeros((mA, nB))
    for i in range(mA):
        for j in range(nB):
            suma = 0
            for k in range(mB):
                suma += A[i][k] * B[k][j]
            res[i][j] = suma 
    return res

def producto_exterior(v: np.array, w: np.array)-> np.array:
    m = len(v)
    n = len(w)

    res = np.zeros((m,n))

    for i in range(m):
        for j in range (n):
            res[i][j] = v[i] * w[j]

    return res    


def QR_con_GS(A: np.array, tol = 1e-12, retorna_nops = False):
    m, n = A.shape
    if m < n:
        return None
    
    Q = np.zeros((m, n)) 
    Q[:, 0] = A[:, 0] / norma(A[:, 0])

    for col in range(1, n):
        Q[:, col] = A[:, col]
        for i in range(col):
            num = producto_interno(Q[:, col], Q[:, i])
            den = producto_interno(Q[:, i], Q[:, i])
            Q[:, col] = Q[:, col] - (num / den) * Q[:, i]
        Q[:, col] = Q[:, col] /norma(Q[:, col])
    
    R = producto_matricial(Q.T, A)
    return Q, R

def QR_con_HH (A: np.array, tol = 1e-12):

    tam = A.shape
    m = tam[0]
    n = tam[1]
    # Chequeo condición
    if m < n:
        return None
    
    Q = np.eye(m)
    R = A.copy()

    for col in range(n):
        # Construyo vector u de la subcolumna
        u = np.zeros(m)
        for i in range(col, m):
            u[i] = R[i][col]
        norma = np.linalg.norm(u)
        if norma < tol:
            continue
        
        signo = np.sign(R[col][col]) if R[col][col] != 0 else 1
        e = np.zeros(m)
        e[col] = 1
        for i in range(m):
            u[i] += signo * norma * e[i]

        uuT = producto_exterior(u, u)
        uTu = producto_interno(u, u)
        H_local = np.eye(m)
        for i in range(m):
            for j in range(m):
                H_local[i][j] -= 2 * uuT[i][j] / uTu

        # Construyo H definitiva
        H = np.eye(m)
        for i in range(col, m):
            for j in range(col, m):
                H[i][j] = H_local[i][j]

        R = producto_matricial(H, R)
        Q = producto_matricial(Q, H)

    return Q, R


def inversa(A):
 
    A = np.array(A, dtype=float)
    n = A.shape[0]
    I = np.eye(n)
    AI = np.concatenate((A, I), axis=1)  # [A | I]

    for k in range(n):
        # Pivoteo parcial
        pivote = np.argmax(np.abs(AI[k:, k])) + k ## busca el indice del valor maximo de la columna k hacia abajo para cambiarlo por el pivote
        if np.isclose(AI[pivote, k], 0.0):
            raise ValueError("La matriz no es invertible.") ## si el maximo es k hcia abajo es 0 entonces la matriz no es invertible

        # Intercambio de filas si es necesario
        if pivote != k:
            AI[[k, pivote], :] = AI[[pivote, k], :] ## swapea filas k y pivote con todas sus columnas

        # Normalizo la fila pivote
        AI[k, :] = AI[k, :] / AI[k, k]

        # Elimino el resto de la columna
        for i in range(n):
            if i != k:
                AI[i, :] = AI[i, :] - AI[i, k] * AI[k, :]

    # La parte derecha de [I | A_inv] es la inversa
    A_inv = AI[:, n:]
    return A_inv

In [9]:
def algoritmo1(X,Y):

    X = np.array(X)
    Y = np.array(Y)


    Xt = transpuesta(X)
    
    n,p = X.shape


    # CASO (a): n > p  ---> usamos pseudo inversa = (X^T X)^(-1)X^T (1)
    if n > p:

        ##xtx = matmul(Xt,X)

        xtx = Xt @ X

        L = cholesky(xtx)
        
        # Acá resolvemos que  (X^T X)U = X^T (2)

        """"

        Wx = Y --> W = Yx^+

        Sacando de (1) U = (X^T X)^(-1)X^T entonce resolvemos el sistema de (2)

        """
        U = pinvEcuacionesNormales(L,Xt)

        # Ahora U = (X^T X)^{-1} X^T  (p x n)

        # W = Y U
        ##W = matmul(Y, U)

        W = Y @ U

    # CASO (b): n < p  ---> usamos pseudo inversa = X^T (XX^T)^(-1) (3)

    elif n < p:


        # Acá asumimos que V(XX^T) = X^T (4)

        """"

        Wx = Y --> W = Yx^+

        Sacando de (4) V(XX^T) = X^T ---> quiero q me quede algo asi (XX^T)V = X^T --> transpongo a ambos lados --> (XX^T)V^T = X --> V^T = (XX^T)^(-1)X es decir q V^T = (X+)^T

        """

        ##xxt = matmul(X,Xt) 
        xxt = X @ Xt
        L = cholesky(xxt)  

        ## Resuelvo V^T del sistema

        Vt =  pinvEcuacionesNormales(L,X)


        #  W = Y V 

        V = transpuesta(Vt)     
        W = Y @ V      
       ## W = matmul(Y, V)

    else:

        X_inv = inversa(X)

        W = Y @ X_inv
        
       ## W = matmul(Y, X_inv) 

    return W





def pinvEcuacionesNormales(L, Y):
          

    Z = sust_adelante_matriz(L, Y)      

    Lt = transpuesta(L)
    U = sust_atras_matriz(Lt, Z)    


    return U

In [ ]:
W_cholesky = algoritmo1(Xt, Yt)
np.save("W_cholesky.npy", np.array(W_cholesky))

W_cholesky = np.load("W_cholesky.npy")

KeyboardInterrupt: 

In [21]:
W_cholesky

array([[1.10599415, 0.93163323, 0.98508831, ..., 1.14009257, 0.90378424,
        1.05089347],
       [1.12371675, 1.16331698, 1.08787736, ..., 1.12618917, 0.97826434,
        0.8883296 ]], shape=(2, 1536))

In [ ]:
### EJERCICIO 4

def pinvHouseHolder(Q,R,Y):

    Rt = transpuesta(R)
    R_inv = inversa(Rt)
    W = matmul( Y , matmul( Q , R_inv ) )
    return W

def pinvGramSchmidt(Q,R,Y):
    Rt = transpuesta(Q)
    R_inv = inversa(Rt)
    W = matmul( Y , matmul( Q , R_inv ) )
    return W


def algoritmo3(X,Y):

    n,p = X.shape

    if rango(X) != n or n <= p:
        return "No se puede descomponer por QR"
    
    elif rango(X) == n and n > p:
        
        Q_hh,R_hh = QR_con_HH(X)

        w_hh = pinvHouseHolder(Q_hh, R_hh, Y)

        Q_gs, R_gs = QR_con_GS(X)

        w_gs = pinvGramSchmidt(Q_gs, R_gs, Y)
        

    return w_hh,w_gs


def matmul_multiple(A,B,C):

    aux = matmul(A,B)

    return matmul(aux,C)
    


In [ ]:
## EJERCICIO 5
def esPseudoInverda(X, pX, tol=1e-8):

   return matricesIguales(matmul_multiple(X,pX,X),X,tol) and matricesIguales(matmul_multiple(pX,X,pX),pX,tol) and matricesIguales(transpuesta(matmul(X,pX)),matmul(X,pX),tol) and matricesIguales(transpuesta(matmul(X,pX)),matmul(pX,X),tol)

## O hacer ultimas 2 con esSimetrica(a) donde a es XpX y en el ultimo a es xpX
   